# Overview
- oputuna についての理解
- suggestion の種類例
    - `trial.suggest_categorical('kernel', ['linear', 'poly', 'rbf'])`
    - `trial.suggest_discrete_uniform('subsample', 0.1, 1.0, 0.1)`
    - `trial.suggest_int('n_estimators', 50, 400)`
    - `trial.suggest_loguniform('c', 1e-5, 1e2)`
    - `trial.suggest_uniform('dropout_rate', 0, 1.0)`

# Import everything I need :)

In [15]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import optuna
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import load_boston
from sklearn.metrics import mean_absolute_error

# Preparation

In [12]:
boston = load_boston()
df = pd.DataFrame(boston.data, columns=boston.feature_names)
target = boston.target

In [13]:
X = df
y = target

# Optuna

In [26]:
# 最小化したい関数
def objective(trial):
    params = {'n_estimators': int(trial.suggest_loguniform('n_estimators', 10, 1000)),
              'max_depth' : trial.suggest_int('max_depth', 5, 300)}
    
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.33, random_state=42)
    model = RandomForestRegressor(**params)
    model.fit(X_train, y_train)
    y_valid_pred = model.predict(X_valid)
    return mean_absolute_error(y_valid, y_valid_pred)

In [31]:
# パラメータ探索
study = optuna.create_study()
study.optimize(objective, n_trials=10)
print(study.best_trial)
print(f'\n best_params = {study.best_params}')

[I 2019-08-31 15:07:21,025] Finished trial#0 resulted in value: 2.1741219294578995. Current best value is 2.1741219294578995 with parameters: {'n_estimators': 273.06962577327, 'max_depth': 14}.
[I 2019-08-31 15:07:21,739] Finished trial#1 resulted in value: 2.1502286883519433. Current best value is 2.1502286883519433 with parameters: {'n_estimators': 398.4130008331471, 'max_depth': 271}.
[I 2019-08-31 15:07:22,504] Finished trial#2 resulted in value: 2.157245225189436. Current best value is 2.1502286883519433 with parameters: {'n_estimators': 398.4130008331471, 'max_depth': 271}.
[I 2019-08-31 15:07:22,773] Finished trial#3 resulted in value: 2.088108398587439. Current best value is 2.088108398587439 with parameters: {'n_estimators': 78.0800124243684, 'max_depth': 187}.
[I 2019-08-31 15:07:22,969] Finished trial#4 resulted in value: 2.1681565440547486. Current best value is 2.088108398587439 with parameters: {'n_estimators': 78.0800124243684, 'max_depth': 187}.
[I 2019-08-31 15:07:23,1

FrozenTrial(number=3, state=<TrialState.COMPLETE: 1>, value=2.088108398587439, datetime_start=datetime.datetime(2019, 8, 31, 15, 7, 22, 508187), datetime_complete=datetime.datetime(2019, 8, 31, 15, 7, 22, 772324), params={'n_estimators': 78.0800124243684, 'max_depth': 187}, distributions={'n_estimators': LogUniformDistribution(low=10, high=1000), 'max_depth': IntUniformDistribution(low=5, high=300)}, user_attrs={}, system_attrs={'_number': 3}, intermediate_values={}, params_in_internal_repr={'n_estimators': 78.0800124243684, 'max_depth': 187.0}, trial_id=3)

 best_params = {'n_estimators': 78.0800124243684, 'max_depth': 187}
